In [1]:
from simple_salesforce import Salesforce
import requests
import pandas as pd
from io import StringIO
import os

sf = Salesforce(
    username=os.getenv('SFDC_USER')
    , password=os.getenv('SFDC_PASS')
    , security_token=os.getenv('SFDC_TOKEN')
)

In [4]:
sf_instance = 'https://crastellopy-dev-ed.develop.my.salesforce.com/' #Your Salesforce Instance URL
reportId = '00Oa5000006XDz7EAG'
export = '?isdtp=p1&export=1&enc=UTF-8&xf=csv'
sfUrl = sf_instance + reportId + export
response = requests.get(sfUrl, headers=sf.headers, cookies={'sid': sf.session_id})
download_report = response.content.decode('utf-8')
separator = "\n\n\n"
try:
    separator_index = download_report.rfind(separator)  # Use rfind to find the *last* occurrence

    if separator_index != -1: #check if the separator was found
        data_part = download_report[:separator_index]
        df1 = pd.read_csv(StringIO(data_part))
    else:
        raise ValueError("Separator not found") # Or handle as you see fit

except ValueError as e:  # Separator not found
    print(f"Error: {e}")  # Print the error message
    df1 = pd.DataFrame()

In [7]:
print(df1.head().to_markdown(floatfmt=''))

|    |   Owner Role | Opportunity Owner   | Account Name    | Opportunity Name             | Stage             | Fiscal Period   |    Amount |   Expected Revenue |   Probability (%) |   Age | Close Date   | Created Date   |   Next Step | Lead Source       | Type                  | Industry   | Forecast Category   |
|---:|-------------:|:--------------------|:----------------|:-----------------------------|:------------------|:----------------|----------:|-------------------:|------------------:|------:|:-------------|:---------------|------------:|:------------------|:----------------------|:-----------|:--------------------|
|  0 |          nan | Irene McCoy         | Morton128 Inc   | Opportunity for Lewis1718    | Closed Won        | Q4-2023         | 1415700.0 |          1415700.0 |               100 |   0.0 | 10/2/2023    | 2/10/2024      |         nan | Employee Referral | New Business          | Energy     | Closed              |
|  1 |          nan | Evelyn Williamson   | Cobb1

In [6]:
df1.head()

,Owner Role,Opportunity Owner,Account Name,Opportunity Name,Stage,Fiscal Period,Amount,Expected Revenue,Probability (%),Age,Close Date,Created Date,Next Step,Lead Source,Type,Industry,Forecast Category
0,NaN,Irene McCoy,Morton128 Inc,Opportunity for Lewis1718,Closed Won,Q4-2023,1415700.0,1415700.0,100,0.0,10/2/2023,2/10/2024,NaN,Employee Referral,New Business,Energy,Closed
1,NaN,Evelyn Williamson,Cobb132 Inc,Opportunity for Freeman746,Closed Won,Q4-2023,1411200.0,1411200.0,100,0.0,11/8/2023,2/10/2024,NaN,Word of mouth,New Business,Energy,Closed
2,NaN,Kelly Frazier,Nichols134 Inc,Opportunity for Klein1486,Qualification,Q3-2024,431073.0,43107.3,10,366.0,7/31/2024,2/10/2024,NaN,Employee Referral,New Business,Energy,Pipeline
3,NaN,Eric Sanchez,Thompson136 Inc,Opportunity for Duncan1725,Qualification,Q3-2024,847450.0,84745.0,10,366.0,7/28/2024,2/10/2024,NaN,Public Relations,Existing Business,Energy,Pipeline
4,NaN,Doroth Gardner,Ramirez137 Inc,Opportunity for Reynolds1553,Value Proposition,Q2-2024,3464115.0,1732057.5,50,366.0,5/25/2024,2/10/2024,NaN,Advertisement,New Business / Add-on,Energy,Pipeline


In [1]:
from simple_salesforce import Salesforce
import requests
import pandas as pd
from io import StringIO
import os
import numpy as np
import tabulate

sf = Salesforce(
    username=os.getenv('SFDC_USER_DEV')
    , password=os.getenv('SFDC_PASS_DEV')
    , security_token=os.getenv('SFDC_TOKEN_DEV')
)

def sf_api_query(soql, dateList=None, tz=None):
    data = sf.query_all(soql)
    try:
        df = pd.DataFrame(data["records"]).drop("attributes", axis=1)
        listColumns = list(df.columns)
        for col in listColumns:
            if any(
                isinstance(df[col].values[i], dict)
                for i in range(0, len(df[col].values))
            ):
                df = pd.concat(
                    [
                        df.drop(columns=[col]),
                        df[col]
                        .apply(pd.Series)
                        .drop("attributes", axis=1)
                        .add_prefix(col + "."),
                    ],
                    axis=1,
                )
                new_columns = np.setdiff1d(df.columns, listColumns)
                for i in new_columns:
                    listColumns.append(i)
        try:
            for date in dateList:
                if max(df[date].str.len()) > 10:
                    try:
                        df[date] = (
                            pd.to_datetime(df[date])
                            .dt.tz_convert(tz)
                            .dt.tz_localize(None)
                        )
                    except:
                        pass
                else:
                    try:
                        df[date] = pd.to_datetime(df[date])
                    except:
                        pass
        except:
            pass
        return df
    except:
        print("The Query returned 0 rows")


soql = '''
    SELECT 
    Id
    ,Name
    ,CloseDate
    ,CreatedDate
    ,Account.Name 
    FROM Opportunity 
    WHERE IsClosed = false 
    LIMIT 20
'''

df = sf_api_query(soql,dateList=['CloseDate','CreatedDate'],tz='America/Chicago')
print(df.head().to_markdown(index=True))

|    | Id                 | Name                       | CloseDate           | CreatedDate         | Account.Name    |
|---:|:-------------------|:---------------------------|:--------------------|:--------------------|:----------------|
|  0 | 006a50000004MCnAAM | Opportunity for Higgins293 | 2024-03-07 00:00:00 | 2024-02-10 02:26:48 | Stone555 Inc    |
|  1 | 006a50000004MowAAE | Opportunity for Parker1715 | 2024-03-08 00:00:00 | 2024-02-10 02:26:48 | Aguilar870 Inc  |
|  2 | 006a50000004MEjAAM | Opportunity for Malone696  | 2024-03-10 00:00:00 | 2024-02-10 02:26:48 | Fletcher429 Inc |
|  3 | 006a50000004MZTAA2 | Opportunity for Murphy1963 | 2024-03-12 00:00:00 | 2024-02-10 02:26:48 | Rivera969 Inc   |
|  4 | 006a50000004MmwAAE | Opportunity for Newton532  | 2024-03-13 00:00:00 | 2024-02-10 02:26:48 | Fletcher234 Inc |


In [6]:
print(df1.head().to_markdown())

|    | Product Name                                            | Product Code                      | Litera Product Family   | Litera Product Group   | Product Line of Business   |   Product Persona |   Product Category |   Is Services Product |   Active (Product) |   One-time |   EOL Date | Case Safe ID       |   NetSuite Id |   NetSuite Id (IO) |
|---:|:--------------------------------------------------------|:----------------------------------|:------------------------|:-----------------------|:---------------------------|------------------:|-------------------:|----------------------:|-------------------:|-----------:|-----------:|:-------------------|--------------:|-------------------:|
|  0 | FileTrail Language Support - French - Hosted            | SW-FT-French-ST-List              | Governance              | FileTrail              | Firm Performance           |               nan |                nan |                     0 |                  1 |          0 |        nan | 01t4X

In [7]:
df1.head().to_clipboard()

In [9]:
df = pd.read_clipboard()

In [10]:
print(df.head().to_markdown())

|    | Product Name   | Product Code   | Product Category   | Product Group   | Line of Business   | Case Safe ID       |
|---:|:---------------|:---------------|:-------------------|:----------------|:-------------------|:-------------------|
|  0 | Core Software  | SW-C           | Software           | Core            | Core               | 01t4X0000012345678 |
|  1 | Pro Software   | SW-P           | Software           | Pro             | Pro                | 01t4X0000012345679 |
|  2 | Implementation | IMP            | Services           | Services        | Services           | 01t4X0000012345680 |
|  3 | Hardware A     | HW-A           | Hardware           | Terminals       | Hardware           | 01t4X0000012345681 |
|  4 | Hardware B     | HW-B           | Hardware           | Printers        | Hardware           | 01t4X0000012345682 |


In [2]:
sf.sf_instance

'crastellopy-dev-ed.develop.my.salesforce.com'